In [2]:
%pwd

'c:\\Users\\ASUS Laptop\\Desktop\\myGymAI\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\ASUS Laptop\\Desktop\\myGymAI'

In [5]:
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Extract text from PDF
def load_pdf(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader,
    )
    documents = loader.load()
    return documents

c:\Anaconda\envs\aienv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
extracted_docs = load_pdf("data")

In [8]:
extracted_docs

[Document(metadata={'producer': 'GPL Ghostscript 9.10', 'creator': '', 'creationdate': '2016-12-23T16:38:21-08:00', 'moddate': '2016-12-23T16:38:21-08:00', 'author': '', 'title': '', 'subject': '', 'keywords': '', 'source': 'data\\Gym_book.pdf', 'total_pages': 584, 'page': 0, 'page_label': 'i'}, page_content='Free ebooks ==>   www.ebook777.com\nENCYCLOPEDIA OF \nMUSCLE & STRENGTH\nJIM STOPPANI, PHD\nHuman Kinetics\nSECOND EDITION'),
 Document(metadata={'producer': 'GPL Ghostscript 9.10', 'creator': '', 'creationdate': '2016-12-23T16:38:21-08:00', 'moddate': '2016-12-23T16:38:21-08:00', 'author': '', 'title': '', 'subject': '', 'keywords': '', 'source': 'data\\Gym_book.pdf', 'total_pages': 584, 'page': 1, 'page_label': 'ii'}, page_content="Free ebooks ==>   www.ebook777.com\nLibrary of Congress Cataloging-in-Publication Data\nStoppani, James, 1968-\n  [Encyclopedia of muscle & strength]\n  Jim Stoppani’s encyclopedia of muscle & strength / Jim Stoppani. -- Second Edition.\n       pages 

In [9]:
from typing import List 
from langchain.schema import Document  

def filter_short_docs(docs: List[Document]) -> List[Document]:
    short_docs: List[Document] = []
    for doc in docs:
        src = doc.metadata.get("source")
        short_docs.append(Document(page_content=doc.page_content, metadata={"source": src}))
    return short_docs

In [10]:
short_docs = filter_short_docs(extracted_docs)

In [11]:
short_docs 

[Document(metadata={'source': 'data\\Gym_book.pdf'}, page_content='Free ebooks ==>   www.ebook777.com\nENCYCLOPEDIA OF \nMUSCLE & STRENGTH\nJIM STOPPANI, PHD\nHuman Kinetics\nSECOND EDITION'),
 Document(metadata={'source': 'data\\Gym_book.pdf'}, page_content="Free ebooks ==>   www.ebook777.com\nLibrary of Congress Cataloging-in-Publication Data\nStoppani, James, 1968-\n  [Encyclopedia of muscle & strength]\n  Jim Stoppani’s encyclopedia of muscle & strength / Jim Stoppani. -- Second Edition.\n       pages cm.\n  Includes bibliographical references and index.\n1.  Weight training. 2.  Bodybuilding. 3.  Muscle strength.  I. Title. II. Title: Jim Stoppani’s encyclopedia of muscle and \nstrength.\n  GV546.S74 2014\n  613.7'13--dc23\n                                                            2014024299\nISBN: 978-1-4504-5974-7 (print)\nCopyright © 2015, 2006 by Jim Stoppani\nAll rights reserved. Except for use in a review, the reproduction or utilization of this work in any form or by any 

In [12]:
# Split text into chunks
def split_docs(docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
    )
    texts_chunked = text_splitter.split_documents(docs)
    return texts_chunked

In [13]:
text_chunked = split_docs(short_docs)

In [14]:
len(text_chunked)

3003

In [15]:
from langchain.embeddings import HuggingFaceEmbeddings

def download_embeddings():
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        )
    return embeddings

embedding = download_embeddings()

C:\Users\ASUS Laptop\AppData\Local\Temp\ipykernel_25456\1099264637.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [16]:
embedding

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [17]:
vector = embedding.embed_query("Hello world")

In [18]:
len(vector)

384

In [19]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = ''

In [58]:
from pinecone import Pinecone
pinecone_api = PINECONE_API_KEY

pc = Pinecone(api_key=pinecone_api)

In [59]:
from pinecone import ServerlessSpec
index_name = "gym-chatbot"
if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    ) 

    index = pc.Index(index_name)

In [41]:
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_documents(documents=text_chunked, embedding=embedding, index_name=index_name)  

KeyboardInterrupt: 

In [60]:
# Load from existing index
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_existing_index(embedding=embedding, index_name=index_name)

In [61]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [62]:
retrieved_docs = retriever.invoke("What is Chest?")
retrieved_docs

[Document(id='de341bd6-ee3a-47ae-8134-e9562f508398', metadata={'source': 'data\\Gym_book.pdf'}, page_content='below chest level.'),
 Document(id='f17bfc62-3fee-4dfa-8602-323cbb5fe506', metadata={'source': 'data\\Gym_book.pdf'}, page_content='Free ebooks ==>   www.ebook777.com\nJim Stoppani’s Encyclopedia of Muscle & Strength54\nChest\nThe chest refers to the muscle group known as the \npectoralis major. This consists of the upper pecto-\nralis major and lower pectoralis major (see figure \n5.1). The pectoralis muscles perform movements \nsuch as horizontal adduction of the upper arms, \nas in the dumbbell fly. For detailed descriptions \nof chest exercises, refer to chapter 14. Basic, \nmultijoint exercises for the chest involve pressing'),
 Document(id='439b3120-2575-4e06-93f4-ca7202f3d82a', metadata={'source': 'data\\Gym_book.pdf'}, page_content='your upper chest.\nPOWER CLEAN')]

In [63]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o-mini")

In [64]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [65]:
system_prompt = ( "You are a helpful AI assistant, expert in gym and body building."
                    "Use the following pieces of context to answer the question at the end."
                    " If you don't know the answer, just say that you don't know, don't try to make up an answer."
                    "\n\n"
                    "{context}")

prompt = ChatPromptTemplate.from_messages(
    [
    ("system", system_prompt),
    ("user", "{input}"),
    ]
)


In [66]:
question_answer_chain = create_stuff_documents_chain(model, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [67]:
response = rag_chain.invoke({"input": "How to train chest? And what should i eat afterwards?"})
print(response["answer"])

To train your chest effectively, you should include exercises that target the upper, middle, and lower pectoralis muscles. A good chest workout might include the following exercises:

1. **Incline Bench Press** (targets upper pectoralis): Start with this exercise as your first movement, aiming for a weight that allows you to complete about 10 reps.
2. **Flat Bench Press** (targets middle pectoralis): This is a staple exercise that allows for heavy lifting and effective muscle building.
3. **Decline Bench Press** (targets lower pectoralis): This exercise can round out your chest workout by ensuring you stimulate the lower part of your chest.

You can also incorporate additional exercises such as dumbbell flyes or cable crossovers for variety.

As for post-workout nutrition, it's important to consume a meal or snack that includes both protein and carbohydrates to aid in recovery and muscle growth. Good options include:

- Protein shake with a banana or oats
- Grilled chicken with brown r